In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

In [8]:
size = 'demo'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/train/history.parquet'
history_train_df = pd.read_parquet(history_train_path)
print(f'Raw {size} train history df shape:          ',history_train_df.shape)

behaviors_train_path = f'./files/parquet/ebnerd_{size}/train/behaviors.parquet'
behaviors_train_df = pd.read_parquet(behaviors_train_path)
print(f'Raw {size} train behaviors df shape:          ',behaviors_train_df.shape)

# Ready embeddings
embeddings_path = f'./files/parquet/Ekstra_Bladet_word2vec/document_vector.parquet'
embeddings_df = pd.read_parquet(embeddings_path).tail(60000)
print(f'embeddings df shape:          ',embeddings_df.shape)

Raw demo train history df shape:           (1590, 5)
Raw demo train behaviors df shape:           (24724, 17)
embeddings df shape:           (60000, 2)


In [9]:
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.max())


print(behaviors_train_df.impression_time.min())
print(behaviors_train_df.impression_time.max())

2023-04-27 07:00:05
2023-05-18 06:59:51
2023-05-18 07:00:03
2023-05-25 06:59:52


In [10]:
embeddings_df.head()

,article_id,document_vector
65541,8220538,"[0.041223563, -0.014775106, 0.016598633, 0.053..."
65542,8220745,"[0.029996565, 0.036494605, 0.003614329, 0.0258..."
65543,8220790,"[0.046301793, -0.003432609, 0.04390633, 0.0488..."
65544,8220803,"[0.063142404, 0.046301603, 0.012075641, 0.0489..."
65545,8220879,"[-0.010365814, 0.017823182, 0.009858246, -0.01..."


In [11]:
history_train_df.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."
2,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24...."
3,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,..."
4,617963,"[2023-04-27T14:42:25.000000, 2023-04-27T14:43:...","[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59..."


In [13]:
embeddings_lst = embeddings_df['article_id'].tolist()

In [14]:
history_articles_lst = list(set(history_train_df['article_id_fixed'].explode().tolist()))

In [15]:
behaviors_inview_articles_lst = list(set(behaviors_train_df['article_ids_inview'].explode().tolist()))

In [16]:
behaviors_clicked_articles_lst = list(set(behaviors_train_df['article_ids_clicked'].explode().tolist()))

In [18]:

print(len(embeddings_lst))
print(len(history_articles_lst))
print(len(behaviors_inview_articles_lst))
print(len(behaviors_clicked_articles_lst))

60000
4247
2478
1114


In [19]:
final_lst =  []
final_lst.extend(history_articles_lst)
final_lst.extend(behaviors_inview_articles_lst)
final_lst.extend(behaviors_clicked_articles_lst)

final_lst = list(set(final_lst))

In [20]:
len(final_lst)

5638

In [27]:
counter =0
for art in tqdm(final_lst):
    if art in embeddings_lst:
        counter += 1

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_3210/1777265541.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for art in tqdm(final_lst):


  0%|          | 0/6204 [00:00<?, ?it/s]

In [28]:
print(counter)

5900


In [21]:
final_lst_df = pd.DataFrame(final_lst, columns=['articles_id'])

In [23]:
final_lst_df

,articles_id
0,9306121
1,9568274
2,9764895
3,5996581
4,9666599
...,...
5633,9764813
5634,9732045
5635,9764822
5636,8617974


In [22]:
# Read cosine similarity matrix pickle file
size = 'demo'
final_lst_path =  f'./files/parquet/ebnerd_{size}/final_lst.pkl'
final_lst_df.to_pickle(final_lst_path)